# Auto-Scan (Step-by-Step) — Microscope + YOLO11

按顺序逐格运行。每一步一个 cell，只有当你运行“开始扫描”那格时才会真正移动/拍照。


In [ ]:
# ===== 0) MACROS / GLOBAL SETTINGS (edit me) =====

# --- 路径 ---
WEIGHTS    = r"E:\\0_FlakesNew\\runs\\train\\AI_BN_1002\\weights\\best.pt"  # YOLO11 .pt
CALIB_TXT  = r"/mnt/data/HQ_pixel_size.txt"                                      # 标定txt：包含 10X/50X 的 um/pixel
OUT_ROOT   = None   # 默认：E:\\Scans\\{date}\\{scan_name}
SCAN_NAME  = None   # 默认：scan_{timestamp}

# --- 灯光 ---
LED_PERCENT = 15   # 连接时设置

# --- 检测默认值（可在后面单独确认） ---
CONF = 0.25
IOU  = 0.45
ALLOW_CLASSES = None   # 例如 ['thin','mid','thick'] 或 None=不限

print("Macros loaded.")


In [ ]:
# ===== 1) CONNECT CONTROLLER =====
from MicroscopeController import MicroscopeController

mc = MicroscopeController()
try:
    mc.set_brightness(LED_PERCENT)
except Exception as e:
    print("Warning: LED set failed:", e)
x0 = float(mc.get_x_position())
y0 = float(mc.get_y_position())
print(f"Connected. Current XY: ({x0:.3f}, {y0:.3f}) mm")


In [ ]:
# ===== 2) CAPTURE START & END =====
# 方案A：手动给定；方案B：自动读取当前/等待你移动到END并回车
START_XY = None  # e.g., (10.0, 15.0)
END_XY   = None  # e.g., (12.0, 17.0)

def capture_start_end(mc):
    if START_XY is not None:
        start_xy = START_XY
    else:
        x = float(mc.get_x_position()); y = float(mc.get_y_position())
        print(f"Captured START from live stage: X={x:.3f}, Y={y:.3f}")
        start_xy = (x, y)

    if END_XY is not None:
        end_xy = END_XY
    else:
        input("Move stage to END point, then press <Enter>...")
        x = float(mc.get_x_position()); y = float(mc.get_y_position())
        print(f"Captured END from live stage: X={x:.3f}, Y={y:.3f}")
        end_xy = (x, y)
    return start_xy, end_xy

start_xy, end_xy = capture_start_end(mc)
print("START_XY:", start_xy, "END_XY:", end_xy)


In [ ]:
# ===== 3) STEP SIZES & GRID PREVIEW =====
# 扫描步距：单位为 mm
STEP_X_MM = 0.20
STEP_Y_MM = 0.20

# 位移后等待的“机械稳定时间”（毫秒）
SETTLE_MS = 50

# 安全Z：如需XY移动前抬高Z，填写数值；否则 None
SAFE_Z    = None

x0, y0 = start_xy
x1, y1 = end_xy
x_min, x_max = sorted([x0, x1])
y_min, y_max = sorted([y0, y1])

n_cols = max(1, int(round((x_max - x_min) / STEP_X_MM)) + 1)
n_rows = max(1, int(round((y_max - y_min) / STEP_Y_MM)) + 1)

print(f"X-range: {x_min:.3f} → {x_max:.3f} mm | step {STEP_X_MM} mm | cols: {n_cols}")
print(f"Y-range: {y_min:.3f} → {y_max:.3f} mm | step {STEP_Y_MM} mm | rows: {n_rows}")
print(f"Settle: {SETTLE_MS} ms   Safe Z:", SAFE_Z)


In [ ]:
# ===== 4) AUTOFOCUS POLICY =====
# 10x 扫描阶段的 AF 频率：每 N 个 tile AF 一次
# - N = 1 表示每个 tile 都 AF（最稳妥）
# - N = 0 表示扫描阶段不 AF
AF_10X_EVERY_N_TILES = 1

# 切到 50x 后是否 AF（建议开）
AF_50X = True

# AF 距离（可微调）
AF_DISTANCE_10X = 0.01
AF_DISTANCE_50X = 0.01

print("AF @10x every N tiles:", AF_10X_EVERY_N_TILES, " (0=off)")
print("AF after switching to 50x:", AF_50X)
print("AF distance 10x:", AF_DISTANCE_10X, " | AF distance 50x:", AF_DISTANCE_50X)


In [ ]:
# ===== 5) DETECTION SETTINGS =====
CONF = CONF
IOU  = IOU
ALLOW_CLASSES = ALLOW_CLASSES  # None 或 ['thin','mid','thick']
print("Weights:", WEIGHTS)
print("Conf:", CONF, "IoU:", IOU, "Allow classes:", ALLOW_CLASSES)


In [ ]:
# ===== 6) DEDUP & CENTERING =====
DEDUP_RADIUS_MM  = 0.10   # 去重半径（mm）；避免重复拍摄相邻候选
CENTER_TOL_MM    = 0.015  # 居中容差（mm）；建议 0.01~0.02
MAX_CENTER_ITERS = 3      # 居中最多迭代次数

print("Dedup radius (mm):", DEDUP_RADIUS_MM)
print("Center tol (mm)  :", CENTER_TOL_MM)
print("Max center iters :", MAX_CENTER_ITERS)


In [ ]:
# ===== 7) CALIBRATION & MAPPING =====
import os

# 如成像坐标与平台坐标有镜像/旋转，可在此指定
FLIP_X = False   # True 表示图像 +x 对应平台 -x
FLIP_Y = False   # True 表示图像 +y 对应平台 -y
ROTATE_DEG = 0.0 # 图像相对平台的旋转角（度）

def load_um_per_px_from_txt(txt_path: str):
    d = {}
    if not os.path.exists(txt_path):
        return d
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s or s.lower().endswith("um/pixel"):
                continue
            if ":" in s:
                k, v = s.split(":", 1)
                try:
                    d[k.strip().upper()] = float(v.strip())
                except:
                    pass
    return d

cal = load_um_per_px_from_txt(CALIB_TXT)
um_per_px_10 = cal.get("10X", 0.35)   # fallback
um_per_px_50 = cal.get("50X", 0.064)  # fallback
px_per_mm_10 = 1000.0 / um_per_px_10
px_per_mm_50 = 1000.0 / um_per_px_50

print("Calibration file:", CALIB_TXT)
print("10x um/px:", um_per_px_10, "=> px/mm:", px_per_mm_10)
print("50x um/px:", um_per_px_50, "=> px/mm:", px_per_mm_50)
print("FlipX:", FLIP_X, "FlipY:", FLIP_Y, "Rotate(deg):", ROTATE_DEG)


In [ ]:
# ===== 8) LIBRARY: Minimal AutoScanner (definition only) =====
from dataclasses import dataclass
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional
import json, math, os, time
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree as KDTree
from ultralytics import YOLO

@dataclass
class CameraMapping:


    
    px_per_mm_10x: float
    px_per_mm_50x: float
    flip_x: bool = False
    flip_y: bool = False
    rotate_deg: float = 0.0
    def img_dxdy_px_to_stage_mm(self, dx_px: float, dy_px: float, use_50x: bool = False):
        px_per_mm = self.px_per_mm_50x if use_50x else self.px_per_mm_10x
        dx_mm = dx_px / px_per_mm
        dy_mm = dy_px / px_per_mm
        if self.flip_x: dx_mm = -dx_mm
        if self.flip_y: dy_mm = -dy_mm
        if abs(self.rotate_deg) > 1e-9:
            th = math.radians(self.rotate_deg)
            c, s = math.cos(th), math.sin(th)
            return dx_mm*c - dy_mm*s, dx_mm*s + dy_mm*c
        return dx_mm, dy_mm

@dataclass
class ScanConfig:
    step_x_mm: float
    step_y_mm: float
    settle_ms: int
    safe_z: Optional[float]
    led_percent: int
    af_10x_every_n_tiles: int   # 0=off, 1=每tile, 2=每2个tile, ...
    af_50x: bool
    af_distance_10x: float
    af_distance_50x: float
    yolo_weights: str
    conf_threshold: float
    iou_threshold: float
    allow_classes: Optional[List[str]]
    dedup_radius_mm: float
    center_tol_mm: float
    max_center_iters: int
    root_out: Optional[str]
    scan_name: Optional[str]
    calib_txt: Optional[str]
    flip_x: bool
    flip_y: bool
    rotate_deg: float

class AutoScanner:
    def __init__(self, mc, cfg: ScanConfig, mapping: CameraMapping):
        self.mc = mc
        self.cfg = cfg
        self.mapping = mapping
        self.model = YOLO(cfg.yolo_weights)
        self.dedup_tree = None
        self.candidate_pts: List[Tuple[float, float]] = []
        self.results_json: Dict[str, Any] = {"tiles": [], "candidates": []}
        self.rows_for_csv: List[Dict[str, Any]] = []

    def _update_dedup_tree(self):
        self.dedup_tree = KDTree(np.array(self.candidate_pts)) if self.candidate_pts else None

    def _is_far(self, x_mm: float, y_mm: float) -> bool:
        if self.dedup_tree is None:
            return True
        d, _ = self.dedup_tree.query([x_mm, y_mm], k=1)
        return d >= self.cfg.dedup_radius_mm

    def _settle(self):
        time.sleep(self.cfg.settle_ms / 1000.0)

    def _goto_xy(self, x, y):
        if self.cfg.safe_z is not None:
            self.mc.set_z_position(self.cfg.safe_z)
        self.mc.set_x_position(x)
        self.mc.set_y_position(y)
        self._settle()

    def _autofocus(self, d) -> bool:
        ok = bool(self.mc.autofocus(d))
        if not ok:
            time.sleep(0.1)
            ok = bool(self.mc.autofocus(d))
        return ok

    def _take_picture(self, path) -> bool:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        return bool(self.mc.take_picture(path))

    def detect(self, img_path):
        r = self.model.predict(img_path, conf=self.cfg.conf_threshold, iou=self.cfg.iou_threshold, verbose=False)
        out = []
        if not r: return out
        r0 = r[0]
        if r0.boxes is None: return out
        names = r0.names
        boxes = r0.boxes.xyxy.cpu().numpy()
        confs = r0.boxes.conf.cpu().numpy()
        clss  = r0.boxes.cls.cpu().numpy().astype(int)
        h, w  = r0.orig_shape
        cx0, cy0 = w/2.0, h/2.0
        for (x1,y1,x2,y2), c, ci in zip(boxes, confs, clss):
            name = names.get(int(ci), str(ci))
            if self.cfg.allow_classes and name not in self.cfg.allow_classes:
                continue
            cx = (x1+x2)/2.0
            cy = (y1+y2)/2.0
            out.append({
                "bbox":[float(x1),float(y1),float(x2),float(y2)],
                "conf":float(c),
                "cls":int(ci),"cls_name":name,
                "cx":float(cx),"cy":float(cy),
                "img_center":[cx0,cy0],"img_size":[w,h]
            })
        return out

    def center_candidate(self, det, current_xy):
        x_mm, y_mm = current_xy
        for _ in range(self.cfg.max_center_iters):
            dx_px = det["cx"] - det["img_center"][0]
            dy_px = det["cy"] - det["img_center"][1]
            dx_mm, dy_mm = self.mapping.img_dxdy_px_to_stage_mm(dx_px, dy_px, use_50x=False)
            nx, ny = x_mm + dx_mm, y_mm + dy_mm
            self._goto_xy(nx, ny)
            x_mm, y_mm = nx, ny
            if (dx_mm**2 + dy_mm**2) ** 0.5 <= self.cfg.center_tol_mm:
                return x_mm, y_mm, True
            tmp = os.path.join(self.out_dir, "tmp", f"recenter_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg")
            self._take_picture(tmp)
            dets = self.detect(tmp)
            if not dets: break
            det = sorted(dets, key=lambda d: (d["cx"]-det["img_center"][0])**2 + (d["cy"]-det["img_center"][1])**2)[0]
        return x_mm, y_mm, False

    def run(self, start_xy, end_xy):
        date_str = datetime.now().strftime("%Y%m%d")
        scan_name = self.cfg.scan_name or f"scan_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        root = self.cfg.root_out or os.path.join("E:\\Scans", date_str, scan_name)
        self.out_dir = root
        tiles = os.path.join(root, "tiles_10x")
        cands = os.path.join(root, "candidates_50x")
        os.makedirs(tiles, exist_ok=True)
        os.makedirs(cands, exist_ok=True)
        os.makedirs(os.path.join(root, "tmp"), exist_ok=True)

        try: self.mc.set_brightness(self.cfg.led_percent)
        except Exception: pass

        self.mc.set_magnification(10)

        x0,y0 = start_xy; x1,y1 = end_xy
        x_min,x_max = sorted([x0,x1]); y_min,y_max = sorted([y0,y1])
        n_cols = max(1, int(round((x_max-x_min)/self.cfg.step_x_mm)) + 1)
        n_rows = max(1, int(round((y_max-y_min)/self.cfg.step_y_mm)) + 1)
        xs = [x_min + c*self.cfg.step_x_mm for c in range(n_cols)]
        ys = [y_min + r*self.cfg.step_y_mm for r in range(n_rows)]

        self.results_json["meta"] = {
            "start_xy":[x0,y0], "end_xy":[x1,y1],
            "x_range":[x_min,x_max], "y_range":[y_min,y_max],
            "n_cols":n_cols, "n_rows":n_rows,
            "step_x_mm":self.cfg.step_x_mm, "step_y_mm":self.cfg.step_y_mm,
            "led_percent":self.cfg.led_percent,
            "af_10x_every_n_tiles": self.cfg.af_10x_every_n_tiles,
            "af_50x": self.cfg.af_50x,
            "dedup_radius_mm": self.cfg.dedup_radius_mm,
            "center_tol_mm": self.cfg.center_tol_mm
        }

        tile_counter = 0
        for ci, x in enumerate(xs):
            col_ys = ys if (ci % 2 == 0) else list(reversed(ys))
            for ri, y in enumerate(col_ys):
                self._goto_xy(x, y)

                if self.cfg.af_10x_every_n_tiles > 0:
                    if (tile_counter % self.cfg.af_10x_every_n_tiles) == 0:
                        self._autofocus(self.cfg.af_distance_10x)

                tile_path = os.path.join(tiles, f"tile_x{ci:03d}_y{ri:03d}_X{x:.3f}_Y{y:.3f}.jpg")
                if not self._take_picture(tile_path):
                    raise RuntimeError(f"Failed to capture 10x tile at X={x:.3f}, Y={y:.3f}")

                dets = self.detect(tile_path)
                self.results_json["tiles"].append({"ci":ci,"ri":ri,"X":x,"Y":y,"path":tile_path,"detections":dets})

                for det in dets:   # 访问全部候选（去重保护）
                    cur_x, cur_y = x, y
                    if self._is_far(cur_x, cur_y):
                        nx, ny, centered_ok = self.center_candidate(det, (cur_x, cur_y))
                        if centered_ok and self._is_far(nx, ny):
                            self.candidate_pts.append((nx,ny)); self._update_dedup_tree()
                            self.mc.set_magnification(50)
                            if self.cfg.af_50x:
                                self._autofocus(self.cfg.af_distance_50x)
                            cand_path = os.path.join(cands, f"cand_X{nx:.3f}_Y{ny:.3f}_cls{det['cls_name']}_conf{det['conf']:.2f}.jpg")
                            ok2 = self._take_picture(cand_path)
                            self.results_json["candidates"].append({
                                "X":nx,"Y":ny, "path": cand_path if ok2 else None,
                                "det":det, "centered":True
                            })
                            self.mc.set_magnification(10)
                tile_counter += 1

        csv_path = os.path.join(root, "scan_log.csv")
        json_path = os.path.join(root, "scan_log.json")
        rows = []
        for t in self.results_json["tiles"]:
            rows.append({"type":"tile","X_mm":t["X"],"Y_mm":t["Y"],"path":t["path"],"n_det":len(t["detections"])})
        for c in self.results_json["candidates"]:
            rows.append({"type":"candidate_50x","X_mm":c["X"],"Y_mm":c["Y"],"path":c["path"],"cls":c.get("det",{}).get("cls_name","")})
        pd.DataFrame(rows).to_csv(csv_path, index=False)
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(self.results_json, f, indent=2)
        print("Saved logs:", csv_path, "\n", json_path)


In [ ]:
# ===== 9) FINAL CONFIRMATION =====
print("Paths:")
print("  WEIGHTS   =", WEIGHTS)
print("  CALIB_TXT =", CALIB_TXT)
print("  OUT_ROOT  =", OUT_ROOT)
print("  SCAN_NAME =", SCAN_NAME)

print("\nStage & grid:")
print("  START_XY  =", start_xy)
print("  END_XY    =", end_xy)
print("  STEP_X_MM =", STEP_X_MM, "STEP_Y_MM =", STEP_Y_MM)
print("  SETTLE_MS =", SETTLE_MS, "SAFE_Z =", SAFE_Z)

print("\nAF:")
print("  AF_10X_EVERY_N_TILES =", AF_10X_EVERY_N_TILES, "| distance10x =", AF_DISTANCE_10X)
print("  AF_50X               =", AF_50X,            "| distance50x =", AF_DISTANCE_50X)

print("\nDetection:")
print("  CONF =", CONF, "IOU =", IOU, "ALLOW_CLASSES =", ALLOW_CLASSES)

print("\nDedup & centering:")
print("  DEDUP_RADIUS_MM  =", DEDUP_RADIUS_MM)
print("  CENTER_TOL_MM    =", CENTER_TOL_MM)
print("  MAX_CENTER_ITERS =", MAX_CENTER_ITERS)

print("\nCalibration:")
print("  FLIP_X =", FLIP_X, "FLIP_Y =", FLIP_Y, "ROTATE_DEG =", ROTATE_DEG)


In [ ]:
# ===== 10) RUN SCAN =====
mapping = CameraMapping(
    px_per_mm_10x = px_per_mm_10,
    px_per_mm_50x = px_per_mm_50,
    flip_x = FLIP_X, flip_y = FLIP_Y, rotate_deg = ROTATE_DEG
)

cfg = ScanConfig(
    step_x_mm=STEP_X_MM, step_y_mm=STEP_Y_MM, settle_ms=SETTLE_MS, safe_z=SAFE_Z,
    led_percent=LED_PERCENT,
    af_10x_every_n_tiles=AF_10X_EVERY_N_TILES,
    af_50x=AF_50X,
    af_distance_10x=AF_DISTANCE_10X, af_distance_50x=AF_DISTANCE_50X,
    yolo_weights=WEIGHTS, conf_threshold=CONF, iou_threshold=IOU,
    allow_classes=ALLOW_CLASSES,
    dedup_radius_mm=DEDUP_RADIUS_MM, center_tol_mm=CENTER_TOL_MM, max_center_iters=MAX_CENTER_ITERS,
    root_out=OUT_ROOT, scan_name=SCAN_NAME,
    calib_txt=CALIB_TXT, flip_x=FLIP_X, flip_y=FLIP_Y, rotate_deg=ROTATE_DEG
)

scanner = AutoScanner(mc, cfg, mapping)
scanner.run(start_xy, end_xy)
print("Scan done. Root:", scanner.out_dir)


In [ ]:
# ===== 11) VIEWER TIP =====
print("若需快速浏览：可运行自带的 scan_viewer.py（若未生成可向我索取），命令示例：")
print("  python scan_viewer.py --root <scan_root>   # <scan_root> 为 scan_log.json 所在目录")


In [ ]:
# ===== 12) CLEANUP =====
try:
    mc.led_off()
except Exception:
    pass
mc.close()
print("Controller closed.")
